# Lab 6 - Tree-based Methods

Dominik Gaweł

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dg7s/Machine-Learning/blob/main/hw/Understanding_Splitting_Criteria_in_CART_for_Regression.ipynb)
-------------------------------

# **Homework Assignment: Understanding Splitting Criteria in CART for Regression**
---------------------

In this assignment, you will explore three common formulations of the splitting criterion used in **CART (Classification and Regression Trees)** for **regression problems**:

1. **Local RSS Minimization**  
2. **RSS Gain Maximization**  
3. **Total RSS Minimization**

You will investigate whether any of these criteria are equivalent, and you will design an experiment to determine which criterion is actually employed in a standard implementation such as **scikit-learn’s DecisionTreeRegressor**.



## **The Problem**

Many treatments of CART for regression describe the split selection process in different ways. Below are three frequently cited formulations. Suppose we have a dataset with features $X$ and target $y$, and we seek to choose a feature $X_j$ and a threshold $t$ to split the data into two regions $R_1(X_j, t)$ and $R_2(X_j, t)$. Denote by $\bar{y}_{R_m}$ the mean of targets within region $R_m$.

1. **Local RSS Minimization**  
   We select the feature and threshold that minimize the **sum of squared errors** in the two resulting child nodes:
   $$
   (X_j^*, t^*) = \arg\min_{X_j, t} \sum_{m=1}^{2} \sum_{i : x_i \in R_m(X_j, t)} (y_i - \bar{y}_{R_m})^2.
   $$

2. **RSS Gain Maximization**  

   It is also a local method, looking only at a parent and two child nodes.

   We select the feature and threshold that maximize the **reduction** in RSS, computed by subtracting the RSS of the two child nodes from the RSS in the parent node:
   $$
   (X_j^*, t^*) = \arg\max_{X_j, t} \Bigl\{
   \underbrace{\sum_{i : x_i \in \text{Parent}} (y_i - \bar{y})^2}_{\text{Parent RSS}}
   \;-\;
   \underbrace{\sum_{m=1}^{2} \sum_{i : x_i \in R_m(X_j, t)} (y_i - \bar{y}_{R_m})^2}_{\text{Children RSS}}
   \Bigr\}.
   $$

3. **Total RSS Minimization**  
   For a dataset $\{(x_i, y_i)\}_{i=1}^N$ with features $X$ and target $y$, let $T$ be the current tree.

   For any split on feature $X_j$ at threshold $t$, define $T(X_j, t)$ as the new tree obtained by splitting one leaf of $T$ into two leaves $R_1(X_j, t)$ and $R_2(X_j, t)$.
   
   Let $\mathrm{Leaves}(T(X_j, t))$ be the set of all leaf indices in this new tree. For each leaf $m \in \mathrm{Leaves}(T(X_j, t))$, define:
   $$
   R_m = \{\, i \,\mid\, x_i \text{ ends in leaf } m\}.
   $$

   $R_m$ set collects all data indices $i$ whose feature vector $x_i$ is classified into the leaf node $m$ when passed through the tree $T(X_j,t)$. In other words, each leaf node $m$ in $T(X_j, t)$ corresponds to a unique path of splits, and any data point $x_i$ that follows that path is assigned to the leaf $m$; hence, it belongs to $R_m$.

   $R_m$ sets for all leafs $m \in \mathrm{Leaves}(T(X_j, t))$ define a partition of all indices.

   Then the objective of **minimizing total Residual Sum of Squares (total RSS)** is stated as:
   $$
   (X_j^*, t^*) = \arg\min_{(X_j, t)} \sum_{m \in \mathrm{Leaves}(T(X_j, t))}
   \sum_{i \in R_m} \Bigl(y_i - \overline{y}_{R_m}\Bigr)^2,
   $$
   where
   $$
   \overline{y}_{R_m} = \frac{1}{\lvert R_m \rvert}
   \sum_{i \in R_m} y_i
   $$
   is the mean response in leaf $m$.


## **Research Questions**

1. **Equivalence Analysis**  
   Determine whether the above formulations are equivalent or if they can yield different split choices. Specifically:
   - Are *local RSS minimization* and *RSS gain maximization* equivalent?
   - Does *total RSS minimization* coincide with either of these two, or is it distinct?
   
2. **Empirical Experiment**  
   Design and conduct a Python experiment to determine which of these formulations is implemented in `scikit-learn` in `DecisionTreeRegressor`. Present numerical results and plots to support your conclusion.


## **Tasks & Deliverables**

1. **Formulation Analysis**  
   - Compare *local RSS minimization*, *RSS gain maximization*, and *total RSS minimization*.
   - If you find that any pair of formulations is equivalent, provide a concise proof.  
   - If you find that they differ, construct a counterexample.

2. **Empirical Verification**  
   - Create a small artificial dataset and train a `DecisionTreeRegressor` from `scikit-learn`.
   - The dataset must be designed in a way that uniquely identifies the formulation used. Provide a short code snippet and a plot or table to support your conclusion.

3. **Report**  
   - Summarize your theoretical insights and empirical findings in a **Colab notebook**.
   - Include the relevant proofs, code, discussion, and conclusions.
   - Place the notebook in your **GitHub repository** for this course, add a link to it in your README.md and add an **“Open in Colab”** badge in the notebook so it can be launched directly.



## Equivalence Analysis

### 1. Local RSS Minimization vs. RSS Gain Maximization for a single node
For a single node (i.e. a local split), the two formulations are equivalent:

\begin{equation}
\arg\max_{X_j, t} \Bigl\{
   \underbrace{\sum_{i : x_i \in \text{Parent}} (y_i - \bar{y})^2}_{\text{Parent RSS}}
   \;-\;
   \underbrace{\sum_{m=1}^{2} \sum_{i : x_i \in R_m(X_j, t)} (y_i - \bar{y}_{R_m})^2}_{\text{Children RSS}}
   \Bigr\} =  \arg\min_{X_j, t} \sum_{m=1}^{2} \sum_{i : x_i \in R_m(X_j, t)} (y_i - \bar{y}_{R_m})^2.
\end{equation}
Since the **Parent RSS** is constant for all candidate splits, maximizing the reduction is equivalent to directly minimizing the **children's RSS**.

---

### 2. Local RSS Minimization vs. RSS Gain Maximization

When building a tree partially—enforced by, for example, setting `max_leaf_nodes`—we do not allow the tree to grow until every leaf is perfectly split. This restriction is necessary in order to highlight differences between the local splitting methods and the global RSS minimization.

Consider the following one-dimensional dataset:

$$
X = (0,\,1,\,2,\,3,\,4), \quad Y = (0,\,1,\,10,\,11,\,12).
$$

We have a first split at $X > 1.5$. This results in:

- **Region 1:**  
  $$ R_1 = \{X_1 = (0,\,1), \quad Y_1 = (0,\,1)\} $$  
  $$ \bar{y}_{R_1} = 0.5,\quad RSS_{R_1} = 0.5 $$
- **Region 2:**  
  $$ R_2 = \{X_2 = (2,\,3,\,4), \quad Y_2 = (10,\,11,\,12)\} $$  
  $$ \bar{y}_{R_2}  = 11,\quad RSS_{R_2} = 2. $$

Now, consider further splitting one of these regions:
- **Splitting Region 1:**With only two observations, we can perfectly split it, reducing its RSS from $0.5$ to $0$.
- **Splitting Region 2:** The best split may only reduce its RSS from $2$ to $0.5$.

**Local RSS minimization** selects the split with the lowest post-split RSS, so Region 1 is optimal (RSS becomes $0$ instead of $0.5$). In contrast, **RSS Gain Maximization** chooses the split with the largest reduction in RSS, favoring Region 2 (gain of $1.5$ versus $0.5$). This illustrates that the methods are not equivalent.

---

### 3. Equivalence of RSS Gain Maximization and Total RSS Minimization


Now we will show that **RSS Gain Maximization** and **Total RSS Minimization** are equivalent. Notice that the difference in Total RSS between the given tree and the tree after a split is given by

$$
\Delta RSS = \text{RSS of the chosen leaf} - \text{(sum of RSS of the children of the chosen leaf)}.
$$

By selecting the leaf whose optimal split minimizes the new Total RSS, we effectively choose the one that produces the largest $\Delta RSS$. In other words, minimizing the Total RSS is equivalent to maximizing the RSS gain. Therefore, the two formulations are equivalent.

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor, export_text

X = np.array([[0], [1], [2], [3], [4]])
y = np.array([0, 1, 10, 11, 12])

reg = DecisionTreeRegressor(max_leaf_nodes=3)
reg.fit(X, y)

tree_rules = export_text(reg, feature_names=["X"])
print(tree_rules)

|--- X <= 1.50
|   |--- value: [0.50]
|--- X >  1.50
|   |--- X <= 2.50
|   |   |--- value: [10.00]
|   |--- X >  2.50
|   |   |--- value: [11.50]



Scikit-learn uses **RSS gain maximization**, as it chooses to split Region 2 to achieve the largest reduction in total RSS.